In [1]:
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
import requests
import json
import re
from multiprocessing.dummy import Pool
import random
import time

In [2]:
init_commenturl = 'https://rate.taobao.com/feedRateList.htm?auctionNumId={}&currentPageNum=1'
init_itemurl = 'https://detail.tmall.com/item.htm?id={}'
comurl =  'https://rate.tmall.com/list_detail_rate.htm?spuId=843588383&sellerId=2631451752&order=3&append=0&tagId=&posi=&picture=&ua=098%23E1hvSpvpvB%2BvUvCkvvvvvjiPPssw0jl8RsMOgjD2PmP9ljtjRLsytjnhPFFpljEhdphvmZCmimLPvhC%2BQIwCvvBvppvviQhvChCvCCptvpvhphvvvvGCvvpvvPMMmphvLhxN2vmFKFEwOv0gQfut8vClpY2heB1ApcHvs8p7regJnZ43IEkZfvHKjrcnnCBQ77QEfwkK5elwfCT9sCOXfvyf8c7yHdUf8B69rfktvpvIphvvvvvvphCvpvA7vvC2GhCvjvUvvhBGphvwv9vvBHBvpCQmvvChx8yCvv3vpvoBnRwnSgyCvvXmp99hVt%2BCvpvVphhvvvvvdphvmZC24kLavHgNiUhCvvsNqMdYtxdNzG9oFYQtvpvhphvvv86CvvDvp6oOiQCJCckCvpvZz%2F%2FhIWt4zYMUQkdGcELzQrGCzWvtvpvhphvvvv%3D%3D&isg=BFFRm1up7l16VwOJjmdSuyW7YF3husQ6zHPypTPjxJlM2n8sew32Ae14eO6ZUl1o&needFold=0'
headers={
       'accept': '*/*',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'zh-CN,zh;q=0.9',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.62 Safari/537.36'
        }

In [3]:
#找商品id列表及评论url列表
def getitemurls(target):
    commenturls = []
    itemulrs = []
    for page in range(1,max_page+1):
        url ='https://s.taobao.com/search?q='+target+itemsort+'&s='+str(44*(page-1))
        res = requests.get(url)
        itemid = re.findall('"nid":"(.*?)","category"',res.text,re.S)
        for i in itemid:
            commenturls.append(init_commenturl.format(i))
            itemulrs.append(init_itemurl.format(i))
    print('商品总数为%d'%len(commenturls))
    return itemulrs

In [4]:
# 保存单个商品评论的函数
def towrtie(info,itemid):
    pd.concat(info,axis=0,ignore_index=True).to_csv(itemid+'commentinfo.csv',header=False)

In [6]:
def geteachpagecom(itemid,pagelist,info):
#设置一个banlist，把被干掉的页面存下来，然后再给自己
    banlist = []
    for eachpage in pagelist:
        t = str(time.time()*1000).split('.')
        params = {
                'currentPage':eachpage,
                 'itemId': itemid,
                 '_ksTS': '%s_%s' % (t[0], t[1]),
                 'callback': 'jsonp%s' %(int(t[1])+1),
                 'content':1
                }
        res = requests.get(comurl,headers=headers,params=params)
        users = []
        comments = []
        jc = json.loads(re.findall(r'{.*}',res.text)[0])
        if 'rateDetail' in jc:
            comments = [i['rateContent'] for i in jc['rateDetail']['rateList']]
            users = [i['displayUserNick'] for i in jc['rateDetail']['rateList']]
            info.append(pd.DataFrame(dict(users=users,comments=comments))) #这样最后一页评论不一样长，不能拼
#            info.append(dict(comments=comments,users=users))
            print('           搞定第%d页'%eachpage)
        else:
#            print('被干掉了等一等')
            banlist.append(eachpage)
            time.sleep(random.uniform(1,6))
    if len(banlist) !=0:
#        print('有%d页没搞定'%len(banlist))
        info.append(geteachpagecom(itemid,banlist,info))
    else:
        #如果执行到这里 那就是都搞定了 开始整理数据       
        #开始保存数据，这个可以先放在外面
        print('全都搞定')
        return info

In [8]:
if __name__ == "__main__":
#   设置最大页数
    max_page = 1
#搜索目标
    target = '行车记录仪'
# 排序关键字，默认为空， 可以按人气、销量排序
    itemsort = '&sort=sale-desc'
#获取商品评论的url的连接
    urls = getitemurls(target)

商品总数为44


In [108]:
# 串行爬取随便玩玩就行
    flag = 1
    result = []
    for item in urls[-5:-1]:
        print('第%d个商品'%flag)
        result.append(getitemcom(item))
        flag += 1

第1个商品
总评论数为553条，共28页
           搞定第1页
           搞定第2页
           搞定第3页
           搞定第4页


From cffi callback <function _verify_callback at 0x000001E43F1E8D90>:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 201, in wrapper
    @wraps(callback)
KeyboardInterrupt


SSLError: ("bad handshake: Error([('SSL routines', 'ssl3_get_server_certificate', 'certificate verify failed')],)",)

In [9]:
#并行
coms =[]
pool = Pool(4)
coms=pool.map(getitemcom,urls)
pool.close()
pool.join()


总评论数为24877条，共99页
总评论数为43048条，共99页
总评论数为3642条，共99页
总评论数为5635条，共99页
           搞定第1页
           搞定第1页
           搞定第1页
           搞定第1页
           搞定第2页
           搞定第2页
           搞定第2页
           搞定第2页
           搞定第3页
           搞定第3页
           搞定第3页
           搞定第3页
           搞定第4页
           搞定第4页
           搞定第4页
           搞定第4页
           搞定第5页
           搞定第5页
           搞定第5页
           搞定第5页
           搞定第6页
           搞定第6页
           搞定第6页
           搞定第6页
           搞定第7页
           搞定第7页
           搞定第7页
           搞定第7页
           搞定第8页
           搞定第8页
           搞定第8页
           搞定第8页
           搞定第9页
           搞定第9页
           搞定第9页
           搞定第9页
           搞定第10页
           搞定第10页
           搞定第10页
           搞定第10页
           搞定第11页
           搞定第11页
           搞定第11页
           搞定第12页
           搞定第12页
           搞定第12页
           搞定第11页
           搞定第13页
           搞定第13页
           搞定第13页
           搞定第12页
           搞定第14页
           搞定第14页
           搞定第13

           搞定第15页
           搞定第18页
           搞定第17页
           搞定第9页
           搞定第16页
           搞定第19页
           搞定第18页
           搞定第17页
           搞定第10页
           搞定第20页
           搞定第19页
           搞定第18页
           搞定第11页
           搞定第21页
           搞定第20页
           搞定第19页
           搞定第22页
           搞定第21页
           搞定第12页
           搞定第20页
           搞定第23页
           搞定第22页
           搞定第13页
           搞定第21页
           搞定第24页
           搞定第23页
           搞定第22页
           搞定第14页
           搞定第25页
           搞定第24页
           搞定第23页
           搞定第26页
           搞定第15页
           搞定第25页
           搞定第24页
           搞定第27页
           搞定第26页
           搞定第16页
           搞定第25页
           搞定第28页
           搞定第27页
           搞定第17页
           搞定第26页
           搞定第29页
           搞定第28页
           搞定第18页
           搞定第27页
           搞定第30页
           搞定第29页
           搞定第19页
           搞定第28页
           搞定第31页
           搞定第30页
           搞定第20页
           搞定第29页
           

           搞定第15页
           搞定第34页
           搞定第35页
           搞定第34页
           搞定第35页
           搞定第36页
           搞定第35页
           搞定第16页
           搞定第36页
           搞定第37页
           搞定第36页
           搞定第17页
           搞定第37页
           搞定第38页
           搞定第37页
           搞定第18页
           搞定第38页
           搞定第39页
           搞定第38页
           搞定第19页
           搞定第39页
           搞定第40页
           搞定第39页
           搞定第20页
           搞定第40页
           搞定第41页
           搞定第40页
           搞定第21页
           搞定第42页
           搞定第41页
           搞定第41页
           搞定第22页
           搞定第43页
           搞定第42页
           搞定第42页
           搞定第23页
           搞定第44页
           搞定第43页
           搞定第43页
           搞定第45页
           搞定第24页
           搞定第44页
           搞定第44页
           搞定第46页
           搞定第25页
           搞定第45页
           搞定第45页
           搞定第47页
           搞定第26页
           搞定第46页
           搞定第46页
           搞定第48页
           搞定第27页
           搞定第47页
           搞定第47页
          

           搞定第96页
           搞定第55页
           搞定第46页
           搞定第24页
           搞定第97页
           搞定第47页
           搞定第56页
           搞定第98页
           搞定第25页
           搞定第57页
           搞定第99页
全都搞定
           搞定第48页
           搞定第26页
           搞定第58页
           搞定第49页
           搞定第27页
总评论数为2936条，共99页
           搞定第59页
           搞定第50页
           搞定第1页
           搞定第28页
           搞定第51页
           搞定第60页
           搞定第2页
           搞定第29页
           搞定第52页
           搞定第61页
           搞定第3页
           搞定第30页
           搞定第53页
           搞定第62页
           搞定第4页
           搞定第54页
           搞定第31页
           搞定第63页
           搞定第5页
           搞定第55页
           搞定第64页
           搞定第32页
           搞定第6页
           搞定第56页
           搞定第65页
           搞定第7页
           搞定第33页
           搞定第66页
           搞定第57页
           搞定第8页
           搞定第34页
           搞定第67页
           搞定第58页
           搞定第9页
           搞定第68页
           搞定第59页
           搞定第35页
           搞定第10页
           搞定第69

           搞定第28页
           搞定第73页
           搞定第15页
           搞定第64页
           搞定第29页
           搞定第74页
           搞定第16页
           搞定第65页
           搞定第30页
           搞定第75页
           搞定第17页
           搞定第66页
           搞定第31页
           搞定第76页
           搞定第18页
           搞定第67页
           搞定第32页
           搞定第77页
           搞定第19页
           搞定第68页
           搞定第33页
           搞定第78页
           搞定第20页
           搞定第69页
           搞定第34页
           搞定第79页
           搞定第21页
           搞定第70页
           搞定第35页
           搞定第80页
           搞定第22页
           搞定第71页
           搞定第36页
           搞定第81页
           搞定第23页
           搞定第72页
           搞定第37页
           搞定第82页
           搞定第24页
           搞定第73页
           搞定第38页
           搞定第83页
           搞定第25页
           搞定第74页
           搞定第39页
           搞定第84页
           搞定第26页
           搞定第75页
           搞定第40页
           搞定第85页
           搞定第27页
           搞定第76页
           搞定第41页
           搞定第86页
           搞定第28页
          

           搞定第5页
           搞定第44页
           搞定第5页
           搞定第6页
           搞定第86页
           搞定第45页
           搞定第6页
           搞定第7页
           搞定第46页
           搞定第87页
           搞定第8页
           搞定第7页
           搞定第47页
           搞定第88页
           搞定第9页
           搞定第48页
           搞定第89页
           搞定第8页
           搞定第10页
           搞定第49页
           搞定第90页
           搞定第9页
           搞定第11页
           搞定第50页
           搞定第91页
           搞定第10页
           搞定第12页
           搞定第51页
           搞定第92页
           搞定第11页
           搞定第13页
           搞定第52页
           搞定第93页
           搞定第14页
           搞定第12页
           搞定第53页
           搞定第94页
           搞定第15页
           搞定第54页
           搞定第13页
           搞定第95页
           搞定第16页
           搞定第55页
           搞定第14页
           搞定第96页
           搞定第17页
           搞定第56页
           搞定第15页
           搞定第18页
           搞定第97页
           搞定第57页
           搞定第16页
           搞定第98页
           搞定第19页
           搞定第58页
           搞定第20页
  

           搞定第12页
           搞定第26页
           搞定第65页
           搞定第94页
           搞定第13页
           搞定第27页
           搞定第66页
           搞定第95页
           搞定第14页
           搞定第28页
           搞定第67页
           搞定第96页
           搞定第15页
           搞定第29页
           搞定第68页
           搞定第97页
           搞定第16页
           搞定第69页
           搞定第30页
           搞定第98页
           搞定第17页
           搞定第70页
           搞定第31页
           搞定第99页
全都搞定
           搞定第18页
           搞定第71页
           搞定第32页
           搞定第19页
           搞定第72页
           搞定第33页
总评论数为9349条，共99页
           搞定第73页
           搞定第34页
           搞定第20页
           搞定第1页
           搞定第35页
           搞定第74页
           搞定第21页
           搞定第2页
           搞定第75页
           搞定第36页
           搞定第22页
           搞定第3页
           搞定第37页
           搞定第76页
           搞定第23页
           搞定第4页
           搞定第38页
           搞定第77页
           搞定第24页
           搞定第5页
           搞定第39页
           搞定第78页
           搞定第25页
           搞定第6页
           搞定

           搞定第41页
           搞定第25页
           搞定第9页
           搞定第28页
           搞定第42页
           搞定第26页
           搞定第29页
           搞定第10页
           搞定第43页
           搞定第27页
           搞定第11页
           搞定第30页
           搞定第44页
           搞定第28页
           搞定第12页
           搞定第31页
           搞定第29页
           搞定第45页
           搞定第13页
           搞定第32页
           搞定第30页
           搞定第46页
           搞定第14页
           搞定第33页
           搞定第31页
           搞定第47页
           搞定第15页
           搞定第34页
           搞定第32页
           搞定第48页
           搞定第16页
           搞定第33页
           搞定第35页
           搞定第49页
           搞定第17页
           搞定第36页
           搞定第34页
           搞定第50页
           搞定第18页
           搞定第51页
           搞定第35页
           搞定第37页
           搞定第19页
           搞定第52页
           搞定第38页
           搞定第36页
           搞定第20页
           搞定第53页
           搞定第39页
           搞定第37页
           搞定第21页
           搞定第40页
           搞定第54页
           搞定第38页
           搞定第22页
           

           搞定第10页
           搞定第63页
           搞定第50页
           搞定第64页
           搞定第11页
           搞定第51页
           搞定第65页
           搞定第52页
           搞定第12页
           搞定第66页
           搞定第53页
           搞定第13页
           搞定第67页
           搞定第54页
           搞定第14页
           搞定第68页
           搞定第55页
           搞定第15页
           搞定第69页
           搞定第56页
           搞定第16页
           搞定第70页
           搞定第57页
           搞定第17页
           搞定第71页
           搞定第58页
           搞定第18页
           搞定第72页
           搞定第59页
           搞定第19页
           搞定第73页
           搞定第60页
           搞定第20页
           搞定第74页
           搞定第61页
           搞定第21页
           搞定第75页
           搞定第22页
           搞定第62页
           搞定第76页
           搞定第63页
           搞定第23页
           搞定第77页
           搞定第64页
全都搞定
           搞定第24页
           搞定第78页
           搞定第25页
总评论数为20164条，共99页
           搞定第79页
           搞定第26页
           搞定第1页
           搞定第80页
           搞定第27页
           搞定第2页
           搞定第81页
        

In [15]:
for i in range(0,len(coms)):
    print('第%d个商品，一共%d条评论'%(i+1,len(coms[i])))

第1个商品，一共1980条评论
第2个商品，一共1980条评论
第3个商品，一共1980条评论
第4个商品，一共1980条评论
第5个商品，一共1980条评论
第6个商品，一共1980条评论
第7个商品，一共1980条评论
第8个商品，一共1980条评论
第9个商品，一共1980条评论
第10个商品，一共1980条评论
第11个商品，一共1980条评论
第12个商品，一共1054条评论
第13个商品，一共1980条评论
第14个商品，一共1980条评论
第15个商品，一共1980条评论
第16个商品，一共1980条评论
第17个商品，一共1980条评论
第18个商品，一共1980条评论
第19个商品，一共1930条评论
第20个商品，一共1980条评论
第21个商品，一共1438条评论
第22个商品，一共1980条评论
第23个商品，一共1980条评论
第24个商品，一共1980条评论
第25个商品，一共472条评论
第26个商品，一共1980条评论
第27个商品，一共1980条评论
第28个商品，一共1980条评论
第29个商品，一共1980条评论
第30个商品，一共1980条评论
第31个商品，一共1980条评论
第32个商品，一共1980条评论
第33个商品，一共1980条评论
第34个商品，一共1980条评论
第35个商品，一共1057条评论
第36个商品，一共1980条评论
第37个商品，一共1980条评论
第38个商品，一共1980条评论
第39个商品，一共1980条评论
第40个商品，一共1261条评论
第41个商品，一共1980条评论
第42个商品，一共361条评论
第43个商品，一共719条评论
第44个商品，一共1980条评论
